In [7]:
%load_ext autoreload
%autoreload 2

In [13]:
import psycopg2
import pandas as pd
# import configparser
import yaml

In [2]:
from sql_postgres import create_staging_sales, drop_staging_sales
from sql_postgres import create_invoices, drop_invoices
from sql_postgres import create_items, drop_items
from sql_postgres import create_product_categories, drop_product_categories
from sql_postgres import create_stores, drop_stores

In [9]:
engine = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="aQAbBqIpcmwVNVtOUt8M",
    host="database-1.ch27epgtqyjq.us-west-2.rds.amazonaws.com",
    port="5432",
)

In [10]:
cur = engine.cursor()

# Explore a small amount of data

For a first guess table schema, use [csvkit](https://csvkit.readthedocs.io/en/latest/index.html)'s `csvstat` and `csvsql`.  For Iowa_Liquor_sales, use a 10000 row subset

````
csvstat subset_of_data.csv
````

CREATE_STAGING_SALES_INITIAL formed from csvsql

In [14]:
with open('data.yml', 'r') as stream:
    data_config = yaml.safe_load(stream)

In [5]:
# data_config = configparser.ConfigParser()
# data_config.read('data.cfg')

['data.cfg']

In [38]:
sales_bucket = data_config["sales_raw"]['bucket']
sales_key = data_config["sales_raw"]["key"]
n_rows = 10000
subset_file = 'subset.csv'

In [39]:
cmd = f"""aws s3api select-object-content --bucket {sales_bucket} \
  --key {sales_key} \
  --expression "select * from s3object limit {n_rows}" \
  --expression-type 'SQL' \
  --input-serialization '{{"CSV": {{}}, "CompressionType": "NONE"}}' \
  --output-serialization '{{"CSV": {{}}}}' "{subset_file}"
"""

In [40]:
! $cmd

In [41]:
!./venv/bin/csvstat $subset_file

  1. "Invoice/Item Number"

	Type of data:          Text
	Contains null values:  False
	Unique values:         9999
	Longest value:         16 characters
	Most common values:    S15295000128 (1x)
	                       S16288500003 (1x)
	                       S09399500011 (1x)
	                       S04545700011 (1x)
	                       S16464700042 (1x)

  2. "Date"

	Type of data:          Date
	Contains null values:  False
	Unique values:         832
	Smallest value:        2012-01-03
	Largest value:         2018-04-11
	Most common values:    2015-10-19 (27x)
	                       2013-05-21 (26x)
	                       2018-04-10 (26x)
	                       2015-04-15 (26x)
	                       2014-11-05 (24x)

  3. "Store Number"

	Type of data:          Number
	Contains null values:  False
	Unique values:         1259
	Smallest value:        2,106
	Largest value:         9,002
	Sum:                   34,875,073
	Mean:                  3,487.856
	Median:           

In [42]:
!./venv/bin/csvsql $subset_file

CREATE TABLE subset (
	"Invoice/Item Number" VARCHAR NOT NULL, 
	"Date" DATE NOT NULL, 
	"Store Number" DECIMAL NOT NULL, 
	"Store Name" VARCHAR NOT NULL, 
	"Address" VARCHAR NOT NULL, 
	"City" VARCHAR NOT NULL, 
	"Zip Code" VARCHAR NOT NULL, 
	"Store Location" VARCHAR, 
	"County Number" DECIMAL, 
	"County" VARCHAR, 
	"Category" DECIMAL, 
	"Category Name" VARCHAR, 
	"Vendor Number" DECIMAL NOT NULL, 
	"Vendor Name" VARCHAR NOT NULL, 
	"Item Number" DECIMAL NOT NULL, 
	"Item Description" VARCHAR NOT NULL, 
	"Pack" DECIMAL NOT NULL, 
	"Bottle Volume (ml)" DECIMAL NOT NULL, 
	"State Bottle Cost" DECIMAL NOT NULL, 
	"State Bottle Retail" DECIMAL NOT NULL, 
	"Bottles Sold" DECIMAL NOT NULL, 
	"Sale (Dollars)" DECIMAL NOT NULL, 
	"Volume Sold (Liters)" DECIMAL NOT NULL, 
	"Volume Sold (Gallons)" DECIMAL NOT NULL
);


In [43]:
df_temp = pd.read_csv(subset_file, nrows=5)
df_temp

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S15295000128,10/23/2013,2524,Hy-Vee Food Store / Dubuque,3500 DODGE ST,DUBUQUE,52001,NaN,31,Dubuque,...,40351,Burnett's Blackberry,12,750,4.17,6.59,4,26.36,3.0,0.79
1,S16288500003,12/16/2013,3740,Hometown Foods / Waterloo,1010 E MITCHELL AVE,WATERLOO,50702,POINT (-92.343118 42.476639),7,Black Hawk,...,64716,Ciroc Peach,12,750,18.75,28.12,24,674.88,18.0,4.76
2,S09399500011,12/11/2012,4495,Casey's General Store #3055 / Grundy,"504, G AVE",GRUNDY CENTER,50638,POINT (-92.770411 42.361494),38,Grundy,...,37934,Skol Vodka,24,375,1.74,2.60,8,20.80,3.0,0.79
3,S04545700011,03/14/2012,3525,Wines and Spirits,106 W 2ND ST,WASHINGTON,52353,POINT (-91.692214 41.300399),92,Washington,...,13388,Northern Light,6,1750,8.32,12.48,6,74.88,10.5,2.77
4,S16464700042,12/23/2013,2638,Hy-Vee Wine and Spirits / Cedar Fall,5925 UNIVERSITY AVE STE A,CEDAR FALLS,50613,POINT (-92.429331 42.512766),7,Black Hawk,...,64601,Caravella Limoncello,6,750,11.25,16.87,6,101.22,4.5,1.19


From this we see that (summarizing only columns that may be of interest in this analysis):

* the invoice/item number is unique and constant length (16 character)
* date range is 2012-01 to 2018-04 and does not contain nulls
* store number is a 4 digit ID and does not contain nulls
* zip code is well formed (always 5 digits) and does not contain nulls
* item number is numeric id up to 6 digits and does not contain nulls
* item description is text up to at least 56 characters and has fewer unique values than item number, suggesting either the schema has changed over time or some different items use the same description
* Category is a numeric ID of 9 digits and contains nulls
* Catagory Name is text of maximum 34 characters (in the subset), contains nulls, and does not have the same number of unique values as category.  This suggests pairings of category names and categories may have changed over time or that some are missing
* vendor numbers are 3 digit numeric IDs and contain nulls
* vendor names contain nulls and the number of unique names does not match unique vendor numbers, suggesting missing items or a change in names over time
* state bottle cost, retail, and sale are all numeric dollar values and contain nulls.  They will be staged as text to avoid rounding and allow their final destination to choose precision

From this, we define a staging table (which takes a direct import of all data in a batch CSV) with schema:

In [48]:
print(CREATE_STAGING_SALES)


CREATE TABLE staging_sales (
  invoice_id VARCHAR NOT NULL, 
  date DATE NOT NULL, 
  store_number "Store Number" DECIMAL NOT NULL, 
  store_name VARCHAR NOT NULL, 
  address VARCHAR NOT NULL, 
  city VARCHAR NOT NULL, 
  zip VARCHAR NOT NULL, 
  store_location VARCHAR, 
  county_number DECIMAL, 
  county VARCHAR, 
  category DECIMAL, 
  category_name VARCHAR, 
  vendor_number DECIMAL NOT NULL, 
  vendor_name VARCHAR NOT NULL, 
  item_number DECIMAL NOT NULL, 
  item_description VARCHAR NOT NULL, 
  pack DECIMAL NOT NULL, 
  bottle_volume_ml DECIMAL NOT NULL, 
  state_bottle_cost DECIMAL NOT NULL, 
  state_bottle_retail DECIMAL NOT NULL, 
  bottles_sold DECIMAL NOT NULL, 
  sale_dollars DECIMAL NOT NULL, 
  volume_sold_liters DECIMAL NOT NULL, 
  volume_sold_gallons DECIMAL NOT NULL
);



And we then define the following OLTP schema for the data we care about.  Note that we only include columns necessary for our current analysis goals - as we will keep the raw data files, additional columns could be loaded later

( ADD SCHEMA HERE )

In [59]:
print(f"{CREATE_STAGING_SALES}\n")
print(f"{CREATE_INVOICES}\n")
print(f"{CREATE_ITEMS}\n")
print(f"{CREATE_PRODUCT_CATEGORIES}\n")
print(f"{CREATE_STORES}\n")


CREATE TABLE staging_sales (
  invoice_id VARCHAR NOT NULL, 
  date DATE NOT NULL, 
  store_id DECIMAL NOT NULL, 
  store_name VARCHAR NOT NULL, 
  address VARCHAR NOT NULL, 
  city VARCHAR NOT NULL, 
  zip VARCHAR NOT NULL, 
  store_location VARCHAR, 
  county_number DECIMAL, 
  county VARCHAR, 
  category_id DECIMAL, 
  category_name VARCHAR, 
  vendor_id DECIMAL NOT NULL, 
  vendor_name VARCHAR NOT NULL, 
  item_id DECIMAL NOT NULL, 
  item_description VARCHAR NOT NULL, 
  pack DECIMAL NOT NULL, 
  bottle_volume_ml DECIMAL NOT NULL, 
  bottle_cost DECIMAL NOT NULL, 
  bottle_retail DECIMAL NOT NULL, 
  bottles_sold DECIMAL NOT NULL, 
  total_sale DECIMAL NOT NULL, 
  volume_sold_liters DECIMAL NOT NULL, 
  volume_sold_gallons DECIMAL NOT NULL
);



CREATE TABLE invoices (
  invoice_id VARCHAR(16) NOT NULL, 
  store_id VARCHAR(4) NOT NULL, 
  date DATE NOT NULL, 
  bottle_cost DECIMAL(7,3) NOT NULL, 
  bottle_retail DECIMAL(7,3) NOT NULL, 
  bottles_sold SMALLINT NOT NULL, 
  total_

# Explore a small amount of weather data

For a first guess table schema, use [csvkit](https://csvkit.readthedocs.io/en/latest/index.html)'s `csvstat` and `csvsql`

````
csvstat subset_of_data.csv
````

In [18]:
bucket = data_config["weather_raw"]['bucket']
key = data_config["weather_raw"]["key"][0]
n_rows = 10000
subset_file = 'subset.csv'

In [19]:
cmd = f"""aws s3api select-object-content --bucket {bucket} \
  --key {key} \
  --expression "select * from s3object limit {n_rows}" \
  --expression-type 'SQL' \
  --input-serialization '{{"CSV": {{}}, "CompressionType": "NONE"}}' \
  --output-serialization '{{"CSV": {{}}}}' "{subset_file}"
"""

In [20]:
! $cmd

# TODO: EXPLAIN CHOICES ON QUERIES.  SHOW QUERIES.  THERES TEXT FILES OF IT ALL

In [21]:
!./venv/bin/csvstat $subset_file

  1. "STATION"

	Type of data:          Text
	Contains null values:  False
	Unique values:         38
	Longest value:         11 characters
	Most common values:    USC00134561 (366x)
	                       USC00138806 (366x)
	                       USC00133239 (366x)
	                       USC00137844 (366x)
	                       USW00094991 (366x)

  2. "NAME"

	Type of data:          Text
	Contains null values:  False
	Unique values:         38
	Longest value:         37 characters
	Most common values:    LAKE PARK, IA US (366x)
	                       WEBSTER CITY, IA US (366x)
	                       GILMAN, IA US (366x)
	                       SPENCER 1 N, IA US (366x)
	                       LAMONI MUNICIPAL AIRPORT, IA US (366x)

  3. "LATITUDE"

	Type of data:          Number
	Contains null values:  False
	Unique values:         38
	Smallest value:        40.622
	Largest value:         43.448
	Sum:                   421,736.276
	Mean:                  42.178
	Median:       

In [22]:
!./venv/bin/csvsql $subset_file

CREATE TABLE subset (
	"STATION" VARCHAR NOT NULL, 
	"NAME" VARCHAR NOT NULL, 
	"LATITUDE" DECIMAL NOT NULL, 
	"LONGITUDE" DECIMAL NOT NULL, 
	"ELEVATION" DECIMAL NOT NULL, 
	"DATE" DATE NOT NULL, 
	"DAPR" DECIMAL, 
	"MDPR" DECIMAL, 
	"PRCP" DECIMAL, 
	"PSUN" BOOLEAN, 
	"SNOW" DECIMAL, 
	"SNWD" DECIMAL, 
	"TAVG" BOOLEAN, 
	"TMAX" DECIMAL, 
	"TMIN" DECIMAL, 
	"TOBS" DECIMAL, 
	"TSUN" BOOLEAN, 
	"WESD" DECIMAL, 
	"WESF" DECIMAL, 
	"WT01" BOOLEAN, 
	"WT02" BOOLEAN, 
	"WT03" BOOLEAN, 
	"WT04" BOOLEAN, 
	"WT05" BOOLEAN, 
	"WT06" BOOLEAN, 
	"WT07" BOOLEAN, 
	"WT08" BOOLEAN, 
	"WT09" BOOLEAN, 
	"WT10" BOOLEAN, 
	"WT11" BOOLEAN, 
	"WT13" BOOLEAN, 
	"WT14" BOOLEAN, 
	"WT15" BOOLEAN, 
	"WT16" BOOLEAN, 
	"WT17" BOOLEAN, 
	"WT18" BOOLEAN, 
	"WT19" BOOLEAN, 
	"WT21" BOOLEAN, 
	"WT22" BOOLEAN
);


In [43]:
df_temp = pd.read_csv(subset_file, nrows=5)
df_temp

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S15295000128,10/23/2013,2524,Hy-Vee Food Store / Dubuque,3500 DODGE ST,DUBUQUE,52001,NaN,31,Dubuque,...,40351,Burnett's Blackberry,12,750,4.17,6.59,4,26.36,3.0,0.79
1,S16288500003,12/16/2013,3740,Hometown Foods / Waterloo,1010 E MITCHELL AVE,WATERLOO,50702,POINT (-92.343118 42.476639),7,Black Hawk,...,64716,Ciroc Peach,12,750,18.75,28.12,24,674.88,18.0,4.76
2,S09399500011,12/11/2012,4495,Casey's General Store #3055 / Grundy,"504, G AVE",GRUNDY CENTER,50638,POINT (-92.770411 42.361494),38,Grundy,...,37934,Skol Vodka,24,375,1.74,2.60,8,20.80,3.0,0.79
3,S04545700011,03/14/2012,3525,Wines and Spirits,106 W 2ND ST,WASHINGTON,52353,POINT (-91.692214 41.300399),92,Washington,...,13388,Northern Light,6,1750,8.32,12.48,6,74.88,10.5,2.77
4,S16464700042,12/23/2013,2638,Hy-Vee Wine and Spirits / Cedar Fall,5925 UNIVERSITY AVE STE A,CEDAR FALLS,50613,POINT (-92.429331 42.512766),7,Black Hawk,...,64601,Caravella Limoncello,6,750,11.25,16.87,6,101.22,4.5,1.19


From this we see that (summarizing only columns that may be of interest in this analysis):

* the invoice/item number is unique and constant length (16 character)
* date range is 2012-01 to 2018-04 and does not contain nulls
* store number is a 4 digit ID and does not contain nulls
* zip code is well formed (always 5 digits) and does not contain nulls
* item number is numeric id up to 6 digits and does not contain nulls
* item description is text up to at least 56 characters and has fewer unique values than item number, suggesting either the schema has changed over time or some different items use the same description
* Category is a numeric ID of 9 digits and contains nulls
* Catagory Name is text of maximum 34 characters (in the subset), contains nulls, and does not have the same number of unique values as category.  This suggests pairings of category names and categories may have changed over time or that some are missing
* vendor numbers are 3 digit numeric IDs and contain nulls
* vendor names contain nulls and the number of unique names does not match unique vendor numbers, suggesting missing items or a change in names over time
* state bottle cost, retail, and sale are all numeric dollar values and contain nulls.  They will be staged as text to avoid rounding and allow their final destination to choose precision

From this, we define a staging table (which takes a direct import of all data in a batch CSV) with schema:

In [48]:
print(CREATE_STAGING_SALES)


CREATE TABLE staging_sales (
  invoice_id VARCHAR NOT NULL, 
  date DATE NOT NULL, 
  store_number "Store Number" DECIMAL NOT NULL, 
  store_name VARCHAR NOT NULL, 
  address VARCHAR NOT NULL, 
  city VARCHAR NOT NULL, 
  zip VARCHAR NOT NULL, 
  store_location VARCHAR, 
  county_number DECIMAL, 
  county VARCHAR, 
  category DECIMAL, 
  category_name VARCHAR, 
  vendor_number DECIMAL NOT NULL, 
  vendor_name VARCHAR NOT NULL, 
  item_number DECIMAL NOT NULL, 
  item_description VARCHAR NOT NULL, 
  pack DECIMAL NOT NULL, 
  bottle_volume_ml DECIMAL NOT NULL, 
  state_bottle_cost DECIMAL NOT NULL, 
  state_bottle_retail DECIMAL NOT NULL, 
  bottles_sold DECIMAL NOT NULL, 
  sale_dollars DECIMAL NOT NULL, 
  volume_sold_liters DECIMAL NOT NULL, 
  volume_sold_gallons DECIMAL NOT NULL
);



And we then define the following OLTP schema for the data we care about.  Note that we only include columns necessary for our current analysis goals - as we will keep the raw data files, additional columns could be loaded later

( ADD SCHEMA HERE )

In [59]:
print(f"{CREATE_STAGING_SALES}\n")
print(f"{CREATE_INVOICES}\n")
print(f"{CREATE_ITEMS}\n")
print(f"{CREATE_PRODUCT_CATEGORIES}\n")
print(f"{CREATE_STORES}\n")


CREATE TABLE staging_sales (
  invoice_id VARCHAR NOT NULL, 
  date DATE NOT NULL, 
  store_id DECIMAL NOT NULL, 
  store_name VARCHAR NOT NULL, 
  address VARCHAR NOT NULL, 
  city VARCHAR NOT NULL, 
  zip VARCHAR NOT NULL, 
  store_location VARCHAR, 
  county_number DECIMAL, 
  county VARCHAR, 
  category_id DECIMAL, 
  category_name VARCHAR, 
  vendor_id DECIMAL NOT NULL, 
  vendor_name VARCHAR NOT NULL, 
  item_id DECIMAL NOT NULL, 
  item_description VARCHAR NOT NULL, 
  pack DECIMAL NOT NULL, 
  bottle_volume_ml DECIMAL NOT NULL, 
  bottle_cost DECIMAL NOT NULL, 
  bottle_retail DECIMAL NOT NULL, 
  bottles_sold DECIMAL NOT NULL, 
  total_sale DECIMAL NOT NULL, 
  volume_sold_liters DECIMAL NOT NULL, 
  volume_sold_gallons DECIMAL NOT NULL
);



CREATE TABLE invoices (
  invoice_id VARCHAR(16) NOT NULL, 
  store_id VARCHAR(4) NOT NULL, 
  date DATE NOT NULL, 
  bottle_cost DECIMAL(7,3) NOT NULL, 
  bottle_retail DECIMAL(7,3) NOT NULL, 
  bottles_sold SMALLINT NOT NULL, 
  total_

# Assumptions/statements

* We will keep all raw data files as a backup.  This allows us to ingest into our DBs only the data we currently plan to use - if additional data was needed in future we could load that as well
* Product/category definitions used in analysis will take the most recent definition as truth.  For example, if in 2012 item `12345` was classified as category `10`, but later item `12345` was classified as category `20`, we will use category `20` for all analyses.  This avoids having to maintain the lineage of any product/category data
* Bottle cost and retail sale price will change over time and thus cannot be an attribute directly in the item table


# Outstanding questions

* do we have weather data for all zips in sales?
* do we have population data for all zips in sales?

# TODO for ETL

* Load product data (item/category) chronologically so we keep only the most recent

In [23]:
from sql_postgres import get_station_latitude_longitude

In [24]:
cur = engine.cursor()
cur.execute(get_station_latitude_longitude)
records = cur.fetchall()

In [33]:
import geopy
geolocator = geopy.Nominatim(user_agent='etl')
zipcode = geolocator.reverse((41.5445, -90.591)).raw['address']['postcode']

In [44]:
def lat_long_to_zip_geo(latitude, longitude):
    location = geolocator.reverse((latitude, longitude))
    
    # Not all locations from this service have zip codes... if nothing, return None
    try:
        return location.raw['address']['postcode']
    except KeyError:
        return None
    
print(lat_long_to_zip(41.5445, -90.591))
print(lat_long_to_zip(43.4022489, -91.2864117))


52804
None


In [42]:
f = lambda t: (t[0], lat_long_to_zip(t[1], t[2]))
list(map(f, records[:3]))

43.401739
-91.28507
43.1409
-91.4926
40.724725
-92.868039


[('US1IAAL0005',
  Location(Allamakee County, Iowa, United States of America, (43.4022489, -91.2864117, 0.0))),
 ('US1IAAL0006',
  Location(672, Woodchuck Drive, Allamakee County, Iowa, 52162, United States of America, (43.14080759357605, -91.49254566651234, 0.0))),
 ('US1IAAP0001',
  Location(1024, South 18th Street, Centerville, Appanoose County, Iowa, 52544, United States of America, (40.7247251744186, -92.86801541860466, 0.0)))]

In [46]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)
search.by_coordinates(43.4022489, -91.2864117, returns=1)[0].zipcode

Start downloading data for simple zipcode database, total size 9MB ...
  1 MB finished ...
  2 MB finished ...
  3 MB finished ...
  4 MB finished ...
  5 MB finished ...
  6 MB finished ...
  7 MB finished ...
  8 MB finished ...
  9 MB finished ...
  10 MB finished ...
  Complete!


In [53]:
search.by_coordinates(43.4022489, -91.2864117, returns=1)[0].zipcode

'52151'

In [60]:
def lat_long_to_zip(latitude, longitude, search=None):
    latitude = float(latitude)
    longitude = float(longitude)
    if not search:
        search = SearchEngine(simple_zipcode=True)
    return search.by_coordinates(latitude, longitude, returns=1)[0].zipcode

In [70]:
search = SearchEngine(simple_zipcode=True)
f = lambda t: (t[0], lat_long_to_zip(t[1], t[2], search))
zipcode_records = map(f, records)

"('US1IAAL0005', '52151'), ('US1IAAL0006', '52156'), ('US1IAAP0001', '52544'), ('US1IAAP0004', '52544'), ('US1IAAP0007', '52544'), ('US1IAAP0008', '52593'), ('US1IAAP0009', '52544'), ('US1IAAP0010', '52581'), ('US1IAAP0011', '52571'), ('US1IABC0002', '52326'), ('US1IABC0005', '52352'), ('US1IABC0006', '50644'), ('US1IABC0008', '50644'), ('US1IABC0009', '50607'), ('US1IABH0003', '50613'), ('US1IABH0006', '50702'), ('US1IABH0007', '50613'), ('US1IABH0008', '50703'), ('US1IABH0010', '50613'), ('US1IABH0015', '50701'), ('US1IABH0017', '50667'), ('US1IABH0018', '50702'), ('US1IABH0019', '50702'), ('US1IABM0002', '50666'), ('US1IABN0003', '50212'), ('US1IABN0005', '50223'), ('US1IABN0009', '50156'), ('US1IABN0012', '50212'), ('US1IABN0013', '50276'), ('US1IABT0011', '52313'), ('US1IABT0012', '52206'), ('US1IABT0013', '52345'), ('US1IABU0002', '50633'), ('US1IABU0003', '50619'), ('US1IABV0001', '50588'), ('US1IABV0002', '51002'), ('US1IACD0005', '52747'), ('US1IACD0007', '52358'), ('US1IACF00

In [65]:
str(list(zipcode_records))

"[('US1IAAL0005', '52151'), ('US1IAAL0006', '52156'), ('US1IAAP0001', '52544'), ('US1IAAP0004', '52544'), ('US1IAAP0007', '52544'), ('US1IAAP0008', '52593'), ('US1IAAP0009', '52544'), ('US1IAAP0010', '52581'), ('US1IAAP0011', '52571'), ('US1IABC0002', '52326'), ('US1IABC0005', '52352'), ('US1IABC0006', '50644'), ('US1IABC0008', '50644'), ('US1IABC0009', '50607'), ('US1IABH0003', '50613'), ('US1IABH0006', '50702'), ('US1IABH0007', '50613'), ('US1IABH0008', '50703'), ('US1IABH0010', '50613'), ('US1IABH0015', '50701'), ('US1IABH0017', '50667'), ('US1IABH0018', '50702'), ('US1IABH0019', '50702'), ('US1IABM0002', '50666'), ('US1IABN0003', '50212'), ('US1IABN0005', '50223'), ('US1IABN0009', '50156'), ('US1IABN0012', '50212'), ('US1IABN0013', '50276'), ('US1IABT0011', '52313'), ('US1IABT0012', '52206'), ('US1IABT0013', '52345'), ('US1IABU0002', '50633'), ('US1IABU0003', '50619'), ('US1IABV0001', '50588'), ('US1IABV0002', '51002'), ('US1IACD0005', '52747'), ('US1IACD0007', '52358'), ('US1IACF0

In [11]:
cur.execute(create_staging_sales)

In [30]:
query = "SELECT * FROM test_iowa_table LIMIT 5"

In [32]:
# # Deleting header which happened to be in table by mistake
# cur.execute("""
# DELETE FROM test_iowa_table 
# WHERE invoice_id in (
#     SELECT invoice_id FROM test_iowa_table LIMIT 1
#     )
# """)

In [17]:
cur.execute(query)
# for t in cur:
#     print(t)

In [21]:
pd.read_sql(query, engine, )

,invoice_id,date,store_number,store_name,address,city,zip,store_location,county_number,county,...,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
0,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
1,S09553800028,12/18/2012,3717,The Liquor Stop / Sumner,201 W 1st ST,SUMNER,50674,POINT (-92.09922100000001 42.847479),09,Bremer,...,36305,Hawkeye Vodka,12,750,3.25,4.88,2,9.76,1.50,0.40
2,S14077800069,10/04/2013,2506,Hy-Vee #1044 / Burlington,3140 AGENCY,BURLINGTON,52601,POINT (-91.136655 40.814666),29,Des Moines,...,64868,Fireball Cinnamon Whiskey,6,1750,15.33,23.00,30,690.00,52.50,13.87
3,S08306600029,10/15/2012,3766,Kolby's Liquor Store,419 W MILWAUKEE,NEW HAMPTON,50659,POINT (-92.32302900000002 43.066752),19,Chickasaw,...,73456,Dekuyper Luscious Razzmatazz,12,750,6.30,9.45,2,18.90,1.50,0.40
4,S11151600002,03/18/2013,2612,Hy-Vee Store / Perry,1215 141 ST,PERRY,50220,POINT (-94.104501 41.830292),25,Dallas,...,11776,Black Velvet,12,750,5.23,7.84,12,94.08,9.00,2.38


In [18]:
pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
1,S09553800028,12/18/2012,3717,The Liquor Stop / Sumner,201 W 1st ST,SUMNER,50674,POINT (-92.09922100000001 42.847479),09,Bremer,...,36305,Hawkeye Vodka,12,750,3.25,4.88,2,9.76,1.50,0.40
2,S14077800069,10/04/2013,2506,Hy-Vee #1044 / Burlington,3140 AGENCY,BURLINGTON,52601,POINT (-91.136655 40.814666),29,Des Moines,...,64868,Fireball Cinnamon Whiskey,6,1750,15.33,23.00,30,690.00,52.50,13.87
3,S08306600029,10/15/2012,3766,Kolby's Liquor Store,419 W MILWAUKEE,NEW HAMPTON,50659,POINT (-92.32302900000002 43.066752),19,Chickasaw,...,73456,Dekuyper Luscious Razzmatazz,12,750,6.30,9.45,2,18.90,1.50,0.40
4,S11151600002,03/18/2013,2612,Hy-Vee Store / Perry,1215 141 ST,PERRY,50220,POINT (-94.104501 41.830292),25,Dallas,...,11776,Black Velvet,12,750,5.23,7.84,12,94.08,9.00,2.38


In [8]:
header = cur.fetchone().split(',')
example

('S09553800028',
 '12/18/2012',
 '3717',
 'The Liquor Stop / Sumner',
 '201 W 1st ST',
 'SUMNER',
 '50674',
 'POINT (-92.09922100000001 42.847479)',
 '09',
 'Bremer',
 '1031080',
 'VODKA 80 PROOF',
 '434',
 'Luxco-St Louis',
 '36305',
 'Hawkeye Vodka',
 '12',
 '750',
 '3.25',
 '4.88',
 '2',
 '9.76',
 '1.50',
 '0.40')